# 케라스와 텐서플로우

## 딥러닝 주요 라이브러리

- 텐서플로우

- 케라스

- 파이토치

- JAX

### 텐서플로우

- 파이썬에 기반한 머신러닝 플랫폼

- 머신러닝 모델의 훈련에 필요한 텐서 연산을 지원
    - 그레이디언트 자동 계산
    - GPU, TPU 등 고성능 병렬 하드웨어 가속기 활용 가능
    - 여러 대의 컴퓨터 또는 클라우드 컴퓨팅 서비스 활용 가능

- C++(게임), 자바스크립트(웹브라우저), TFLite(모바일 장치) 등과 호환 가능

- 단순한 패키지 기능을 넘어서는 머신러닝 플랫폼
    - TF-Agents: 강화학습 연구 지원
    - TFX: 머신러닝 프로젝트 운영 지원
    - TensorFlow-Hub: 사전 훈련된 머신러닝 모델 제공

### 케라스

- 딥러닝 모델 구성 및 훈련에 효율적으로 사용될 수 있는 다양한 수준의 API를 제공
- 텐서플로우의 프론트엔드<font size='2'>front end</font> 인터페이스 기능 수행

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/keras_and_tf.png" style="width:600px;"></div>

### 딥러닝 주요 라이브러리 약력

- 2007년: 씨아노<font size='2'>Theano</font> 공개. 
    텐서를 이용한 계산 그래프, 미분 자동화 등을 최초로 지원한 딥러닝 라이브러리.
- 2015년 3월: 케라스 라이브러리 공개. Theano를 백앤드로 사용하는 고수준 패키지.
- 2015년 11월: 텐서플로우 라이브러리 공개.
- 2016년: 텐서플로우가 케라스의 기본 백엔드로 지정됨.
- 2016년 9월: 페이스북이 개발한 파이토치<font size='2'>PyTorch</font> 공개.
- 2017년: Theano, 텐서플로우, CNTK(마이크로소프트), MXNet(아마존)이 케라스의 백엔드로 지원됨.
    현재 Theano, CNTK 등은 더 이상 개발되지 않으며, MXNet은 아마존에서만 주로 사용됨.
- 2018년 3월: PyTorch와 Caffe2를 합친 PyTorch 출시(페이스북과 마이크로소프트의 협업)
- 2019년 9월: 텐서플로우 2.0부터 케라스가 텐서플로우의 최상위 프레임워크로 지정됨.
- 2023년 가을: Keras Core가 케라스 3.0으로 출시 예정. 텐서플로우, PyTorch, JAX의 프론트엔드 기능 지원.

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch03-keras-core.png?raw=true" style="width:500px;"></div>

## 텐서플로우 텐서

- `tf.Tensor` 자료형
    - 상수 텐서
    - 입출력 데이터 등 변하지 않는 값을 다룰 때 사용.
    - 불변 자료형

- `tf.Variable` 자료형
    - 변수 텐서
    - 모델의 가중치, 편향 등 항목의 업데이트가 필요할 때 사용되는 텐서.
    - 가변 자료형

### 상수 텐서

다양한 방식으로 상수 텐서를 생성할 수 있다.

```python
>>> x = tf.constant([[1., 2.], [3., 4.]])
>>> print(x)
tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
```

```python
>>> x = tf.ones(shape=(2, 1))
>>> print(x)
tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
```

```python
>>> x = tf.zeros(shape=(2, 1))
>>> print(x)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)
```

- `normal()` 함수: 정규 분포 활용

```python
>>> x = tf.random.normal(shape=(3, 1), mean=0., stddev=1.)
>>> print(x)
tf.Tensor(
[[-0.5644841 ]
 [-0.76016265]
 [ 0.30502525]], shape=(3, 1), dtype=float32)
```

- `uniform()` 함수: 균등 분포 활용

```python
>>> x = tf.random.uniform(shape=(3, 1), minval=0., maxval=1.)
>>> print(x)
tf.Tensor(
[[0.33661604]
 [0.09824598]
 [0.32487237]], shape=(3, 1), dtype=float32)
```

**상수 텐서의 수정 불가능성**

```python
>>> x[0, 0] = 1.0
TypeError                                 Traceback (most recent call last)
<ipython-input-7-242a5d4d3c4a> in <module>
----> 1 x[0, 0] = 1.0

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment
```

**텐서 항목의 자료형**

텐서 항목의 자료형은 `EagerTensor` 라는 텐서다.

```python
>>> type(x[0, 0])
tensorflow.python.framework.ops.EagerTensor
```

### 변수 텐서

```python
>>> v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
>>> print(v)
<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-1.3837979 ],
       [-0.23704937],
       [-0.9790895 ]], dtype=float32)>
```

**변수 텐서 교체**

`assign()` 메서드는 해당 텐서를 통채로 다른 텐서로 대체한다.

```python
>>> v.assign(tf.ones((3, 1)))
<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>
```

단, 대체하는 텐서의 모양(shape)이 기존 텐서의 모양과 동일해야 한다.

```python
>>> v.assign(tf.ones((3, 2)))
ValueError                                Traceback (most recent call last)
<ipython-input-13-e381ab0c94e6> in <module>
----> 1 v.assign(tf.ones((3, 2)))

~\anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py in assign(self, value, use_locking, name, read_value)
    886         else:
    887           tensor_name = " " + str(self.name)
--> 888         raise ValueError(
    889             ("Cannot assign to variable%s due to variable shape %s and value "
    890              "shape %s are incompatible") %

ValueError: Cannot assign to variable Variable:0 due to variable shape (3, 1) and value shape (3, 2) are incompatible
```

**변수 텐서 항목 수정**

```python
>>> v[0, 0].assign(3.)
<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>
```

### `assign_add()`와 `assign_sub()`

- `assign_sub()` 메서드는 `-=` 연산자 
- `assign_add()` 메서드는 `+=` 연산자

```python
>>> v.assign_sub(tf.ones((3, 1)))
<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[2.],
       [0.],
       [0.]], dtype=float32)>
```

## 텐서플로우 활용법 기초

### 그레이디언트 테이프

$$
f(x) = x^2 \quad \Longrightarrow \quad \nabla f(x) = \frac{df(x)}{dx} = 2x
$$

```python
>>> input_var = tf.Variable(initial_value=3.)

>>> with tf.GradientTape() as tape:
...     result = tf.square(input_var)

>>> gradient = tape.gradient(result, input_var)

>>> print(gradient)
tf.Tensor(6.0, shape=(), dtype=float32)
```

### 예제: 선형 분류기

**1단계: 데이터셋 생성**

```python
num_samples_per_class = 1000

# 음성 데이터셋
negative_samples = np.random.multivariate_normal(
    mean=[0, 3], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class)

# 양성 데이터셋
positive_samples = np.random.multivariate_normal(
    mean=[3, 0], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class)
```

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/03-07.png" style="width:300px;"></div>

- `negative_sample`: (1000, 2) 모양의 텐서
- `positive_sample`: (1000, 2) 모양의 텐서
- `inputs = np.vstack(negative_sample, positive_sample)`: (2000, 2) 모양의 텐서
    - `negative_sample` 데이터셋이 0번부터 999번까지 인덱스.
    - `positive_sample` 데이터셋이 1000번부터 1999번까지 인덱스.

```python
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
```

음성 샘플의 레이블은 0, 양성 샘플의 레이블은 1로 지정한다.

- `np.zeros((num_samples_per_class, 1), dtype="float32")`: (1000, 1) 모양의 어레이. 0으로 채워짐. 
    0번부터 999번 인덱스까지의 모든 음성 데이터의 타깃은 0임.
- `np.ones((num_samples_per_class, 1), dtype="float32")`: (1000, 1) 모양의 어레이. 1로 채워짐. 
    999번부터 1999번 인덱스까지의 모든 양성 데이터의 타깃은 1임.
- `targets`: (2000, 1) 모양의 어레이.

```python
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype="float32"),
                     np.ones((num_samples_per_class, 1), dtype="float32")))
```

**2단계: 선형 회귀 모델 훈련에 필요한 가중치 변수 텐서와 편향 변수 텐서 생성**

선형 분류기 모델의 예측값 계산은 다음과 같이 아핀 변환으로 이뤄진다.

```python
inputs @ W + b
```

- `inputs`: (2000, 2) 모양의 입력 데이터셋 행렬
- `W`: (2, 1) 모양의 가중치 행렬
- `inputs @ W`: (2000, 1) 모양의 행렬
-  `b`: (1,) 모양의 편향 벡터
- `inputs @ W + b`: (2000, 1) 모양의 출력값 행렬. 즉, 2000 개의 입력 데이터 각각에 대해 하나의 값의 계산됨.

```python
input_dim = 2     # 입력 샘플의 특성이 2개
output_dim = 1    # 각각의 입력 샘플에 대해 하나의 부동소수점을 예측값으로 계산
```

```python
# 가중치: (2, 1) 모양의 가중치 행렬을 균등분포를 이용한 무작위 초기화
W = tf.Variable(initial_value=tf.random.uniform(shape=(input_dim, output_dim)))

# 편향: (1,) 모양의 벡터를 0으로 초기화
b = tf.Variable(initial_value=tf.zeros(shape=(output_dim,)))
```

**3단계: 모델 선언(포워드 패스)**

```python
def dense(inputs, W, b, activation=None):
    outputs = tf.matmul(inputs, W) + b
    if activation != None:
        return activation(outputs)
    else:
        return outputs
```

```python
def model(inputs):
    outputs = dense(inputs, W, b)
    return outputs
```

**4단계: 손실 함수 지정**

$$
\frac{1}{m_b}\sum (y - \hat y)^2
$$

```python
def square_loss(targets, predictions):
    per_sample_losses = tf.square(targets - predictions)
    return tf.reduce_mean(per_sample_losses)
```

**5단계: 훈련 스텝(역전파) 지정**

```python
def training_step(inputs, targets):
    """
    - inputs: 입력 데이터 배치
    - targets: 타깃 배치
    """

    # 손실 함수의 그레이디언트 계산 준비
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = square_loss(targets, predictions)

    # 가중치와 편향에 대한 손실 함수의 그레이디언트 계산
    grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W, b])
    
    # 가중치 행렬과 편향 벡터 업데이트
    W.assign_sub(grad_loss_wrt_W * learning_rate) # 가중치 행렬 업데이트
    b.assign_sub(grad_loss_wrt_b * learning_rate) # 편향 업데이트
    
    return loss
```

**6단계: 훈련 루프 지정**

- 반복해서 훈련한 내용을 출력
- 설명을 간단하게 하기 위해 전체 데이터셋을 하나의 배치로 사용하는 훈련 구현

```python
for step in range(40):
    loss = training_step(inputs, targets)
    print(f"Loss at step {step}: {loss:.4f}")
```

**7단계: 결정경계**

결정경계를 직선으로 그리려면 아래 일차 함수를 이용한다.

```python
y = - W[0] /  W[1] * x + (0.5 - b) / W[1]
```

이유는 아래 식으로 계산되는 모델의 예측값이
0.5보다 큰지 여부에 따라 양성/음성이 판단되기 때문이다.

```python
W[0]*x + W[1]*y + b
```

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch03-bin_classification.png?raw=true" style="width:300px;"></div>

## 케라스 신경망 모델의 핵심 API

- 신경망 모델은 층으로 구성됨

- 모델에 사용되는 층의 종류와 층을 쌓는 방식에 따라 모델이 처리할 수 있는 데이터와 훈련 방식이 달라짐

- 케라스 라이브러리가 층을 구성하고 훈련 방식을 관장하는 다양한 API 제공

### 층

- 입력 데이터를 지정된 방식에 따라 다른 모양의 데이터로 변환하는 **포워드 패스**<font size='2'>forward pass</font> 담당

- 데이터 변환에 사용되는 가중치<font size='2'>weight</font>와 편향<font size='2'>bias</font> 저장

**층의 종류**

층의 종류에 따라 입력 배치 데이터셋 텐서의 모양이 달라진다.

- `Dense` 클래스
    - 밀집층 생성
    - `(배치 크기, 특성 수)` 모양의 2D 텐서로 입력된 데이터셋 처리.

- `LSTM` 또는 `Conv1D` 클래스
    - 순차 데이터와 시계열 데이터 분석에 사용되는 순환층 생성
    - `(배치 크기, 타임스텝 수, 특성 수)` 모양의 3D 텐서로 입력된 순차 데이터셋 처리.

- `Conv2D` 클래스
    - 합성곱 신경망(CNN) 구성에 사용되는 합성곱층 생성
    - `(배치 크기, 가로, 세로, 채널 수)` 모양의 4D 텐서로 제공된 이미지 데이터셋 처리.

**`tf.keras.layers.Layer` 클래스**

- 케라스의 모든 층 클래스는 `tf.keras.layers.Layer` 클래스를 상속한다.

- 상속되는 `__call__()` 메서드: 
    - 가중치와 편향 텐서의 초기화. 가중치와 편향이 이미 생성되어 있다면 새로 생성하지 않고 그대로 사용.
    - 입력 데이터셋을 출력 데이터셋으로 변환하는 포워드 패스를 수행    

- `__call__()` 메서드가 하는 일

```python
def __call__(self, inputs):
    if not self.built:
        self.build(inputs.shape)
        self.built = True
    return self.call(inputs)
```

**`Dense` 클래스 직접 구현하기**

{numref}`%s절 <sec:nn-mnist>`에서 MNIST 데이터셋을 이용한 분류 모델에 사용된
신경망 모델은 연속으로 쌓은 두 개의 `Dense` 층으로 구성된다.

```python
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])
```

`Dense` 클래스와 유사하게 작동하는 클래스를 직접 정의하려면 
상속해야 하는 `keras.layers.Layer` 클래스의 `__call()__` 메서드에 의해 호출되는
`build()` 메서드와 `call()` 메서드를 구현해야 한다.
아래 `SimpleDense` 클래스가 `Dense` 클래스의 기능을 단순화하여 구현한다.

두 메서드의 정의에 사용된 매개변수와 메서드는 다음과 같다.

- `units`: 출력 샘플의 특성 수 지정
- `activation`: 활성화 함수 지정
- `input_shape`: 입력값(`inputs`)으로 얻은 입력 배치의 2D 모양 정보. 둘째 항목이 입력 샘플의 특성 수.
- `add_weight(모양, 초기화방법)`: 지정된 모양의 텐서 생성 및 초기화. `Layer` 클래스에서 상속.

```python
from tensorflow import keras

class SimpleDense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units           # 유닛 개수 지정
        self.activation = activation # 활성화 함수 지정

    # 가중치와 편향 초기화
    def build(self, input_shape):
        input_dim = input_shape[-1]   # 입력 샘플의 특성 수
        self.W = self.add_weight(shape=(input_dim, self.units),
                                 initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros")

    # 데이터 변환(포워드 패스)
    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y
```

:::{prf:example} `SimpleDense` 층의 데이터 변환
:label: simpledense

모델 훈련 과정에서 포워드 패스는 층에서 층으로 이어지는 연속된 데이터 변환으로 이뤄진다.
`SimpleDense` 층을 이용하여 입력 데이터셋이 어떻게 변환되어 다음 층으로 전달되는 과정을 살펴본다.

아래 코드에서 `my_dense` 변수는 하나의 `SimpleDense` 층을 가리킨다.

- 유닛 수: 32개
- 활성화 함수: `relu`

```python
>>> my_dense = SimpleDense(units=512, activation=tf.nn.relu)
```

아래 코드는 입력 배치 데이터셋으로 사용할 (128, 784) 모양의 텐서를 생성한다.

- 128: 배치 크기
- 784: MNIST 데이터셋의 손글씨 이미지 한 장의 특성 수(`28 * 28 = 128`)

```python
>>> input_tensor = tf.ones(shape=(128, 784))
```

이제 `my_dense`를 함수 호출하듯이 사용하면 출력값이 계산된다.
즉, 포워드 패스가 실행된다.
층은 입렵 데이터셋을 처리할 때 입력 데이터셋의 모양을 확인하기에 
굳이 입력 데이터셋에 대한 정보를 미리 요구하지 않는다.


```python
>>> output_tensor = my_dense(input_tensor)
```

내부적으로는 `__call__()` 메서드가 호출되어 다음 사항들이 연속적으로 처리된다. 

- 가중치 텐서와 와 편향 텐서가 생성되지 않은 경우
    - `(784, 512)` 모양의 가중치 텐서 `W` 생성 및 무작위 초기화. 782는 입력 샘플의 특성 수, 512는 층의 유닛 수.
    - `(512, )` 모양의 편향 텐서 `b` 생성 및 `0`으로 초기화. 512는 층의 유닛 수.
    - 포워드 패스: 생성된 가중치와 편향을 이용하여 출력값 계산.

- 가중치 텐서와 와 편향 텐서가 생성되어 있는 경우. 즉 훈련이 반복되는 경우.
    - 포워드 패스: 역전파로 업데이트된 가중치와 편향을 이용하여 출력값 계산.

층의 출력값은 `(128, 32)` 모양의 텐서다.
이유는 각 데이터 샘플의 784개의 특성이 32개의 특성으로 변환되었기 때문이다.

```python
>>> print(output_tensor.shape)
(128, 512)
```
:::

### 모델

앞서 살펴 본 `Sequential` 모델은 층을 일렬로 쌓는 모델이며
각각의 층은 이전 층에서 전달된 배치 데이터셋을 변환해서 다음 층으로 전달한다.

**`tf.keras.Model` 클래스**

`Sequential` 클래스를 포함하여 케라스에서 지원되는 모든 모델 클래스는 `tf.keras.Model` 클래스를 상속한다.
예를 들어 `Sequential` 클래스를 이용하여 정의된 MNIST 분류 모델을 `SimpleDense` 층을 이용하여 직접 다음과 같이 정의할 수 있다.

```python
class MyMNistModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_1 = SimpleDense(units=512, activation=tf.nn.relu)   # 첫째 밀집층
        self.layer_2 = SimpleDense(units=10, activation=tf.nn.softmax) # 둘째 밀집층

    # 포워드 패스: 층과 층을 연결하는 방식으로 구현
    def call(self, inputs):
        features = self.layer_1(inputs)
        outputs = self.layer_2(features)
        return outputs
```

:::{admonition} 직접 구현한 `SimpleDense` 층과 케라스의 `Dense` 층의 차이점
:class: note

`keras.layers.Dense` 층을 이용한다면 다음과 같이 활성화 함수를 문자열로 지정할 수 있다.

```python
class MyMNistModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_1 = keras.layers.Dense(units=512, activation="relu")
        self.layer_2 = keras.layers.Dense(units=10, activation="softmax")

    def call(self, inputs):
        features = self.layer_1(inputs)
        outputs = self.layer_2(features)
        return outputs
```
:::

**모델을 하나의 층으로 활용하기**

기존에 정의된 모델을 다른 모델을 구성할 때 하나의 층으로 활용할 수도 있다.
이런 이유로 `tf.keras.Model` 클래스는 `tf.keras.layers.Layer` 클래스를 
상속하도록 설계되어 있다.
`tf.keras.Model` 클래스의 활용법에 대한 보다 자세한 설명은 {numref}`%s장 <ch:working_with_keras>`을 참고한다.

**모델의 학습과정과 층의 구성**


모델의 학습과정은 전적으로 층의 구성방식에 의존한다. 
그리고 층의 구성 방식은 주어진 데이터셋과 모델이 해결해야 하는 문제에 따라 달라진다.
층을 구성할 때 특별히 정해진 규칙은 없지만 
문제 유형에 따른 권장 모델이 다양하게 개발되어 있다.

앞으로 보다 복잡하고 다양한 방식으로 층을 구성하는 방식들을 살펴볼 것이다.
예를 들어, 아래 그림은 {numref}`%s장 자연어 처리 <ch:nlp>`에서 소개하는
트랜스포머<font size='2'>Transformer</font> 모델의 복잡한 층 연결 구조를 보여준다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/transformer0001.png" style="width:500px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

**모델 컴파일**

선언된 모델을 훈련시키려면 다음 세 가지 설정을 추가로 지정해야 한다.

- 손실 함수
    - 훈련 중 모델의 성능이 얼마나 나쁜지 측정.
    - 가중치와 편향 의존하는 함수
    - 가중치와 편향에 대해 미분 가능해야 함.
    - 옵티마이저가 역전파를 통해 모델의 성능을 향상시키는 방향으로 모델의 가중치를 업데이트할 때 참고하는 함수임.
- 옵티마이저
    - 가중치와 편향을 업데이트하는 역전파 반복 실행
- 평가지표 
    - 훈련과 테스트 과정을 모니터링 할 때 사용되는 모델 평가 지표.
    - 손실 함수와는 달리 훈련에 사용되지 않음.
    - 단순히 모델 성능 평가에 사용됨.

케라스를 이용하면 옵티마이저, 손실 함수, 평가지표를 문자열로 지정할 수 있다.

```python
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer="rmsprop",
              loss="mean_squared_error",
              metrics=["accuracy"])
```

각각의 문자열은 특정 파이썬 객체를 가리킨다.

| 문자열 | 파이썬 객체 |
| :--- | :--- |
| `"rmsprop"` | `keras.optimizers.RMSprop()` |
| `"mean_squared_error"` | `keras.losses.MeanSquaredError()` |
| `"accuracy"` | `keras.metrics.BinaryAccuracy()]` |

따라서 지정된 문자열을 사용하는 대신 파이썬 객체를 직접 지정해도 된다.
만약 사용자가 직접 구현한 클래스의 객체를 이용하려면
앞서 `SimpleDense`를 통해 본 것처럼 적절한 클래스를 상속하면서
동시에 필수 메서드를 모두 적절하게 재정의해야<font size='2'>overriding</font> 한다.

```python
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])
```

다음 두 가지의 경우엔 문자열 대신 해당 객체를 지정해야 한다.
- 예를 들어, 기본값과 다른 학습률(`learning_rate`)을 사용하는 옵티마이저를 지정하는 경우
- 사용자가 직접 정의한 객체를 사용하는 경우

아래 코드는 직접 객체를 지정하는 방식으로 모델을 컴파일하는 형식을 보여준다.
```python
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
              loss=사용자정의손실함수객체,
              metrics=[사용자정의평가지표_1, 사용자정의평가지표_2])
```

일반적으로 가장 많이 사용되는 옵티마이저, 손실함수, 평가지표는 다음과 같으며
앞으로 다양한 예제를 통해 적절한 옵티마이저, 손실함수, 평가지표를 선택하는 방법을 살펴볼 것이다.

옵티마이저:

- SGD (with or without momentum)
- RMSprop
- Adam
- Adagrad

손실 함수:

- CategoricalCrossentropy
- SparseCategoricalCrossentropy
- BinaryCrossentropy
- MeanSquaredError
- KLDivergence
- CosineSimilarity

평가지표:

- CategoricalAccuracy
- SparseCategoricalAccuracy
- BinaryAccuracy
- AUC
- Precision
- Recall

### 훈련 루프

모델을 컴파일한 다음에 `fit()` 메서드를 호출하면
모델은 스텝 단위로 반복되는 **훈련 루프**<font size='2'>training loop</font>가 작동한다.
지정된 에포크 만큼 또는 학습이 충분히 이루어졌다는 평가가 내려질 때까지
훈련을 반복한다.

**모델 훈련**

모델을 훈련시키려면 `fit()` 메서드를 적절한 인자들과 함께 호출해야 한다.

```python
training_history = model.fit(
    inputs,
    targets,
    epochs=5,
    batch_size=128
)
```

- (지도 학습 모델의 경우) 훈련셋(inputs)과 타깃셋(targets): 보통 넘파이 어레이 또는 텐서플로우의 `Dataset` 객체 사용
- 에포크(`epochs`): 전체 훈련 세트를 몇 번 훈련할 지 지정
- 배치 크기(`batch_size`): 하나의 스텝 과정에서 사용되는 데이터 묶음(배치)의 크기

**`History` 객체: 훈련 결과**

모델의 훈련 결과로 `History` 객체가 반환된다.
예를 들어 `History` 객체의 `history` 속성은 에포크별로 계산된 손실값과 평가지표값을
사전 자료형으로 가리킨다.

```python
>>> training_history.history
{'loss': [9.07500171661377,
  8.722702980041504,
  8.423994064331055,
  8.137178421020508,
  7.8575215339660645],
 'binary_accuracy': [0.07800000160932541,
  0.07999999821186066,
  0.08049999922513962,
  0.08449999988079071,
  0.0860000029206276]}
```

**검증 데이터 활용**

머신러닝 모델 훈련의 목표는 훈련셋에 대한 높은 성능이 아니라
훈련에서 보지 못한 새로운 데이터에 대한 정확한 예측이다.
훈련 중에 또는 훈련이 끝난 후에 모델이 새로운 데이터에 대해 정확한 예측을 하는지
여부를 판단하도록 할 수 있다.

이를 위해 전체 데이터셋을 훈련셋과 검증셋<font size='2'>validation dataset</font>으로 구분한다.
훈련셋과 검증셋의 비율은 보통 8대2 또는 7대3 정도로 하지만
훈련셋이 매우 크다면 검증셋의 비율을 보다 적게 잡을 수 있다.
훈련셋 자체가 매우 작은 경우엔 검증셋을 따로 분리하기 보다는 K-겹 교차 검증 등을 사용해야 한다.

훈련셋과 검증셋이 서로 겹치지 않도록 주의해야 한다.
그렇지 않으면 훈련 중에 모델이 검증셋에 포함된 데이터를 학습하기에
정확환 모델 평가를 할 수 없게 된다.

*훈련 중 모델 검증*

아래 코드는 미리 지정된 검증셋 `val_inputs`와 검증 타깃값 `val_targets`를
`validation_data`의 키워드 인자로 지정해서
모델 훈련 중에 에포크 단위로 측정하도록 한다.

```python
model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets)
)
```

*훈련 후 모델 검증*

훈련이 끝난 모델의 성능 검증하려면 `evaluate()` 메서드를 이용한다.
배치 크기(`batch_size`)를 지정하여 배치 단위로 학습하도록 한다.

```python
>>> loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)
```

반환값으로 지정된 손실값과 평가지표를 담은 리스트가 생성된다.

```python
>>> print(loss_and_metrics)
[0.29411643743515015, 0.5333333611488342]
```

### 예측

모델의 훈련과 검증이 완료되면 실전에서 새로운 데이터에 대한 예측을 진행한다.
데이터셋에 포함된 모든 데이터에 대한 예측을 한 번에 실행할 수 있으며
두 가지 방식이 존재한다.

**모델 적용**

모델을 마치 함수처럼 이용한다. 

```python
predictions = model(new_inputs)
```

내부적으론 앞서 설명한 `__call()__` 메서드가 실행된다.
따라서 `call()` 메서드를 사용하는 포워드 패스가 실행되어
예측값이 계산된다.

하지만 이 방식은 입력 데이터셋 전체를 대상으로 한 번에 계산하기에
데이터셋이 너무 크면 계산이 너무 오래 걸리거나 메모리가 부족해질 수 있다.
따라서 배치를 활용하는 `predict()` 메서드를 활용할 것을 추천한다.

**`predict()` 메서드**

훈련된 모델의 `predict()` 메서드는 배치 크기를 지정하면
배치 단위로 예측값을 계산한다.

```python
predictions = model.predict(new_inputs, batch_size=128)
```

## 연습 문제

1. [(실습) 케라스와 텐서플로우](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/excs/exc-keras_and_tf.ipynb)